In [42]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import MCMT
import time
import hashlib
from math import sqrt, pi

def classical_pow(target_zeros, max_nonce):
    steps = 0
    for nonce in range(max_nonce):
        steps += 1
        hash_result = hashlib.sha256(str(nonce).encode()).hexdigest()
        if hash_result.startswith('0' * target_zeros):
            return nonce, hash_result, steps
    return None, None, steps

def grovers_circuit(num_qubits, target_state, num_iterations):
    qc = QuantumCircuit(num_qubits, num_qubits)  # Add a classical register for measurements

    # Initialize in the uniform superposition
    qc.h(range(num_qubits))

    # Apply Grover's algorithm iterations
    for _ in range(num_iterations):
        # Oracle: flips the sign of the target state
        qc.h(target_state)
        qc.z(target_state)
        qc.h(target_state)

        # Diffuser
        qc.h(range(num_qubits))
        qc.x(range(num_qubits))
        qc.h(num_qubits - 1)
        qc.append(MCMT('x', num_ctrl_qubits=num_qubits - 1, num_target_qubits=1), range(num_qubits))
        qc.h(num_qubits - 1)
        qc.x(range(num_qubits))
        qc.h(range(num_qubits))

    # Measurement
    qc.measure(range(num_qubits), range(num_qubits))

    return qc

def quantum_pow(target_zeros, max_nonce, num_qubits):
    # Find a target state that has the specified number of leading zeros
    target_state = 0
    for i in range(2**num_qubits):
        binary_rep = format(i, f'0{num_qubits}b')
        if binary_rep.startswith('0' * target_zeros):
            target_state = i
            break

    # Estimate the number of solutions and the number of Grover iterations
    num_solutions = 2**(num_qubits - target_zeros)
    num_iterations = int((pi / 4) * sqrt(2**num_qubits / num_solutions))

    # Construct a quantum circuit for Grover's algorithm
    qc = grovers_circuit(num_qubits, target_state, num_iterations)
    
    # Simulate the quantum circuit
    simulator = AerSimulator()
    transpiled_circuit = transpile(qc, simulator)
    result = simulator.run(transpiled_circuit, shots=1).result()
    
    # Extract the result and convert to a nonce
    nonce = int(result.get_counts().most_frequent(), 2)
    return nonce, hashlib.sha256(str(nonce).encode()).hexdigest(), num_iterations

# Example usage
target_zeros = 7
max_nonce = 2**20
num_qubits = 20

nonce, hash_result, quantum_steps = quantum_pow(target_zeros, max_nonce, num_qubits)
print(f"Quantum PoW: Nonce = {nonce}, Hash = {hash_result}, Steps = {quantum_steps}")

# Classical PoW
nonce, hash_result, classical_steps = classical_pow(target_zeros, max_nonce)
print(f"Classical PoW: Nonce = {nonce}, Hash = {hash_result}, Steps = {classical_steps}")

# Compare steps
speedup = classical_steps / quantum_steps
print(f"Speedup (in terms of steps): {speedup:.2f}x")


Quantum PoW: Nonce = 482800, Hash = 3f38a47633ebad836dae4e993db4652d60bbb9c7e13c2968efaaaba64769382d, Steps = 8
Classical PoW: Nonce = 665782, Hash = 0000000399c6aea5ad0c709a9bc331a3ed6494702bd1d129d8c817a0257a1462, Steps = 665783
Speedup (in terms of steps): 83222.88x
